In [73]:
import glob, os, sys, re, shutil
import hashlib

from PIL.ExifTags import TAGS
from PIL import Image

from datetime import datetime
from dateutil import parser

from optparse import OptionParser

In [137]:
# Read all the EXIF tags
def get_exif(fn):
    reDT = re.compile(r'DateTime')
    DTkeys = ['DateTimeOriginal','DateTimeDigitized''DateTime']
    ret = {}
    m = ''
    t = []
    i = Image.open(fn)
    info = i._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            ret[decoded] = value
            if decoded == 'Model':
                m = "".join([c for c in value if c.isalpha() or c.isdigit() or c in ',. ']).rstrip()
        tk = [k for k in ret if k in DTkeys][0]
        if tk:
            t = datetime.strptime(ret[tk],'%Y:%m:%d %H:%M:%S')
        return ret,t,m

# MD5 of a file
fmd5 = lambda f: hashlib.md5(open(f,'rb').read()).hexdigest()

In [136]:
# Make a directory
def makeDir(newDir,newDirIsFile=True):
    if newDir[0] == os.path.sep:
        newDirBld = os.path.sep
    else:
        newDirBld = ''            
    if not newDirIsFile:
        newFN = newFN+os.path.sep
    for dirPart in os.path.split(
                        os.path.abspath(
                        os.path.expanduser(newDir)))[0].split(os.path.sep):
        newDirBld = os.path.join(newDirBld,dirPart)
        if not os.path.isdir(newDirBld):
            os.mkdir(newDirBld,0750)
        
# Move file to a new name and location, creating path as needed
def fileMove(oldFN,newFN,copy=True,verbose=True):
    duplicateFile = False
    fileExt = 0
    newFNParts = os.path.splitext(f)
    while os.path.isfile(newFN)and not duplicateFile:
        if fmd5(oldFN) == fmd5(newFN):
            duplicateFile = True
        else:
            newFN = ('_%03d' % fileExt).join(os.path.splitext(f))
            fileExt+=1
            
    if not duplicateFile:
        makeDir(newFN)
        if copy:
            shutil.copy(oldFN,newFN)
        else:
            shutil.move(oldFN,newFN)
    elif verbose:
        print oldFN + ' is the same as' + newFN

In [ ]:
rootdir = os.path.abspath(os.path.expanduser('~/Dropbox/Camera Uploads'))
if not os.path.isdir(rootdir):
    rootdir = os.path.abspath(
        os.path.expanduser('/Storage/Media/Pictures/SORTED/'))
    sortdir = os.path.abspath(
        os.path.expanduser('/Storage/Media/Pictures/NEWSORT'))
ownerTAG = 'CVL'

reDT = re.compile(r'DateTime')
DTkeys = ['DateTimeOriginal','DateTimeDigitized','DateTime']

# Walk a directory
for root, subFolders, fileNames in os.walk(rootdir):
    for fileName in fileNames:
        if re.match('.*\.jpg',fileName):
            FN = os.path.join(root,fileName)
            md5base = fmd5(FN)
            exifTags,t,m = get_exif(FN)
            if not t or not m:
                #DTkeys = [ x for x in exifTags.keys() if reDT.match(str(x)) ]
                print FN + ' not sortable'
            else:
                nFN = os.path.join(sortdir,m,t.strftime('%Y/%m/%y%m%d-%H%M%S_'+ownerTAG+'.jpg'))
                fileMove(FN,nFN,verbose=False)


'Canon DIGITAL IXUS 400'